In [10]:

#need to go this route https://www.springeropen.com/get-published/indexing-archiving-and-access-to-data/rss-feeds


In [5]:
import threading
import requests
import pandas as pd
from metapub import PubMedFetcher
from Bio import Entrez
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

Entrez.email = "epovlot8589@gmail.com"
Entrez.api_key = '8f0801462285bdfc3696dd0c55c09e52cd08'


def get_pubmed(days_back):
    def pubmed_affiliations(pmid):
        soup = BeautifulSoup(requests.get(f'https://pubmed.ncbi.nlm.nih.gov/{pmid}/long-authors/').text, 'html.parser')
        ul_element = soup.find('ul', class_='item-list')
        li_elements = ul_element.find_all('li')
        return [li.get_text(strip=True)[1:] for li in li_elements]
    def process_pubmed_ids(start, end):
        for i in range(start, end):
            pubmed_id = pubmed_ids[i]
            paper_info = process_pubmed_id(pubmed_id)
            if paper_info != None:
                out.append(paper_info)
    def process_pubmed_id(pubmed_id):
        try:
            paper_info = {}
            article = fetch.article_by_pmid(pubmed_id)
            paper_info['abstract'] = article.abstract
            paper_info['authors'] = article.authors_str
            paper_info['title'] = article.title
            paper_info['subject'] = article.journal
            paper_info['paper_id'] = article.doi
            paper_info['date'] = article.history['medline'].strftime('%Y-%m-%d')
            paper_info['affiliations'] = pubmed_affiliations(pubmed_id)
            return paper_info
        except:
            return
    days_ago = datetime.now() - timedelta(days=days_back)
    print(days_ago)
    fetch = PubMedFetcher()
    search_query = f"({days_ago.strftime('%Y/%m/%d')}[Date - Publication] : {datetime.now().strftime('%Y/%m/%d')}[Date - Publication])"

    batch_size = 250
    start_idx = 0
    pubmed_ids = []
    while True:
        handle = Entrez.esearch(db="pubmed", term=search_query, retmax=batch_size, retstart=start_idx)
        record = Entrez.read(handle)
        handle.close()
        
        batch_ids = record["IdList"]
        if not batch_ids:
            break  # No more results
        
        pubmed_ids.extend(batch_ids)
        start_idx += batch_size

    rate_limit = 10 
    out = []

    max_threads = len(pubmed_ids) if len(pubmed_ids) < rate_limit else rate_limit
    batch_size = len(pubmed_ids) // max_threads

    threads = []
    for i in range(max_threads):
        start = i * batch_size
        end = (i + 1) * batch_size if i < max_threads - 1 else len(pubmed_ids)
        thread = threading.Thread(target=process_pubmed_ids, args=(start, end))
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()
    out = [i for i in out if i is not None]

    return pd.DataFrame.from_records(out)


In [6]:
df = get_pubmed(2)
df

2023-10-04 16:17:23.657003


In [4]:
fetch = PubMedFetcher()
days_ago = datetime.now() - timedelta(days=2)
search_query = f"({days_ago.strftime('%Y/%m/%d')}[Date - Publication] : {datetime.now().strftime('%Y/%m/%d')}[Date - Publication])"
batch_size = 250
start_idx = 0
pubmed_ids = []
handle = Entrez.esearch(db="pubmed", term=search_query, retmax=batch_size, retstart=start_idx)
record = Entrez.read(handle)
record

{'Count': '7700', 'RetMax': '250', 'RetStart': '0', 'IdList': ['37801365', '37801364', '37801363', '37801362', '37801361', '37801360', '37801359', '37801358', '37801357', '37801356', '37801355', '37801354', '37801353', '37801352', '37801351', '37801350', '37801349', '37801348', '37801347', '37801346', '37801345', '37801343', '37801342', '37801341', '37801340', '37801337', '37801335', '37801333', '37801332', '37801331', '37801330', '37801329', '37801328', '37801327', '37801326', '37801325', '37801324', '37801323', '37801322', '37801321', '37801311', '37801310', '37801309', '37801308', '37801307', '37801306', '37801305', '37801304', '37801303', '37801299', '37801298', '37801296', '37801295', '37801294', '37801293', '37801292', '37801291', '37801290', '37801289', '37801288', '37801287', '37801286', '37801285', '37801284', '37801283', '37801282', '37801281', '37801280', '37801279', '37801278', '37801277', '37801276', '37801275', '37801274', '37801273', '37801272', '37801271', '37801270', '

In [19]:
df[df['affiliations'].astype(str).str.contains('Emory')]

,abstract,authors,title,subject,paper_id,date,affiliations
77,Rules are needed for human research in commerc...,Rahimzadeh V; Fogarty J; Caulfield T; Auñón-Ch...,Ethically cleared to launch?,Science,10.1126/science.adh9028,2023-09-28,"[Center for Medical Ethics and Health Policy, ..."
277,None,Drexler K; Edens EL; Trafton JA; Compton WM,In the balance: No new diagnosis needed in add...,Addiction,10.1111/add.16349,2023-09-28,[Department of Psychiatry and Behavioral Scien...
304,This work proposes a novel generative multimod...,Geenjaar EPT; Lewis NL; Fedorov A; Wu L; Ford ...,Chromatic fusion: Generative multimodal neuroi...,Hum Brain Mapp,10.1002/hbm.26479,2023-09-27,[School of Electrical and Computer Engineering...
311,IMPORTANCE: Although several clinician- and pa...,Mastacouris N; Tannenbaum R; Strunk A; Koptyev...,Outcome Measures for the Evaluation of Treatme...,JAMA Dermatol,10.1001/jamadermatol.2023.3282,2023-09-27,"[Northwell Health, New Hyde Park, New York., D..."
363,OBJECTIVE: While modern antiretroviral therapy...,Parra-Rodriguez L; O'halloran J; Wang Y; Jin W...,Common antiretroviral combinations are associa...,AIDS,10.1097/QAD.0000000000003730,2023-09-29,"[Department of Medicine, Washington University..."
452,INTRODUCTION: Immunotherapy has revolutionized...,Tarekegn K; Keskinkilic M; Kristoff TJ; Evans ...,The role of immune checkpoint inhibition in tr...,Expert Rev Anticancer Ther,10.1080/14737140.2023.2265059,2023-09-29,"[Winship Cancer Institute, Emory University, A..."
743,OBJECTIVE: Surgically assisted rapid palatal e...,Jara SM; Thuler ER; Hutz MJ; Yu JL; Cheong CS;...,Posterior Palatal Expansion via Subnasal Endos...,Laryngoscope,10.1002/lary.31060,2023-09-29,"[Division of Sleep Surgery, Department of Otor..."
816,Magnetic resonance imaging (MRI) is increasing...,Lehto TK; Pylväläinen J; Sandeman K; Kenttämie...,Histomic and transcriptomic features of MRI-vi...,Int J Cancer,10.1002/ijc.34743,2023-09-28,"[Department of Pathology, University of Helsin..."
855,Although offspring of women exposed to childho...,McKenna BG; Knight AK; Smith AK; Corwin EJ; Ca...,Infant epigenetic aging moderates the link bet...,Dev Psychopathol,10.1017/S0954579423001232,2023-09-29,"[Department of Psychology, Emory University, A..."
875,BACKGROUND: Advances in face and neck lifting ...,Talei B; Shauly O; Marxen T; Menon A; Gould DJ,The Mastoid Crevasse and 3-Dimensional Conside...,Aesthet Surg J,10.1093/asj/sjad321,2023-09-28,"[Beverly Hills, CA, USA., Department of Surger..."
